# Processamento da Fonte de Dados

Este notebook realiza o processo de ETL (extract, transform, load) para as fontes de dados disponibilizadas pelo [Mapa das Organizações da Sociedade Civil](https://mapaosc.ipea.gov.br/base-dados).

## Configurando o Ambiente

Para manutenção da simplicidade, os códigos necessários para realizar o processamento dos dados estão segmentados em módulos. Este caderno tem o propósito apenas de orquestrar e exibir os resultados dos processamentos realizados por tais módulos.

Para isto, é necessário reconfigurar o ambiente do notebook, para que os caminhos dos módulos sejam resolvidos corretamente, como em uma aplicação convencional Python.


In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Added to Python path: {project_root}")
print(f"Current working directory: {os.getcwd()}")

## Etapas do Processamento

Este caderno executa as seguintes etapas para produzir uma base de dados tratada:

### 1. Conversão de Encoding

Os datasets disponibilizados encontram-se em diversos formatos, além de contarem com "encodings" inadequados para a leitura e análise adequada através de algoritmos.

Para cada dataset original, esta etapa gera fontes contendo encoding adequado (em utf8) para a realização de etapas posteriores.


In [ ]:
import pandas
from data.processing.data_parser import to_utf8

ONGS_DATASET = input(
    "Enter the ONGS dataset filename (with extension): ").strip()

result_path = to_utf8(ONGS_DATASET)

if result_path is None:
    raise FileNotFoundError(
        f"File {ONGS_DATASET} not found or could not be converted to UTF-8.")

reencoded_dataset = pandas.read_csv(result_path, sep=";", encoding="utf-8")
reencoded_dataset.info()

### 2. Tratamento de Organizações

A segunda fase lida com as informações disponibilizadas na base `osc_2025_2.csv`, que contém informações sobre as OSCs catalogadas na fonte mencionada.

#### 2.1. Tratamento de Áreas de Atuação

O dataset original armazena os códigos de área de atuação de cada OSC de maneira tabular, conforme o exemplo abaixo:

| Cnpj           | ... | Area_x | Area_y | Area_z |
| -------------- | --- | ------ | ------ | ------ |
| 00000000000000 | ... | 0      | 1      | 0      |
| 11111111111111 | ... | 1      | 0      | 1      |

Onde valores `1` indicam que a OSC faz parte da área de atuação correspondente à coluna.

Para melhorar a legibilidade do dataset final, esta estrutura é trauzida e concatenada em um único texto descritivo, conforme o exemplo abaixo:

| Cnpj           | ... | Areas de Atuação |
| -------------- | --- | ---------------- |
| 00000000000000 | ... | Area y           |
| 11111111111111 | ... | Area x, Area z   |

In [ ]:
from data.processing.data_parser import osc_dataset

osc_df = osc_dataset(reencoded_dataset)
osc_df.loc[:20]